In [1]:
import random
import numpy as np
import itertools
import copy

## Define a linear function for f

In [45]:
n = 10
t = 300
x = np.random.uniform(0,1, n)
a = np.random.uniform(-1,1, n)

In [46]:
x

array([0.39258359, 0.17741823, 0.59250329, 0.54281811, 0.33530203,
       0.62242382, 0.31806745, 0.12839349, 0.88517413, 0.24545588])

In [47]:
a

array([ 0.22190737, -0.70470618, -0.28948519,  0.35585917,  0.44527886,
       -0.74862312, -0.30095507, -0.87786712, -0.1669726 , -0.95652113])

In [48]:
def f_linear(x):   
    return np.dot(a,x.T)

f_linear(x)

-0.9239426147152634

## Approximation for Multi-Linear Extension

In [49]:
x_sample = 0

def F_unfixed(x, f, t, x_sample): 
    # Here, take x_sample as a parameter for the ease of 
    # using both the F and F_fixed functions in this framework

    sum_R = 0

    for i in range(t):

        x_sample = np.random.uniform(0,1, x.shape)

        R_t = x >= x_sample

        sum_R = sum_R + f(R_t)

    return sum_R/t

# In Function F, re-sample from x every time we run the function
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))
print(F_unfixed(x, f_linear, 100, x_sample))

-0.7980966155975172
-0.99250439092805
-0.9334419022178256


In [50]:
# Sample from x (stored in x_sample) once and for all  
# x_sample defined here will be used as a parameter for the code below

def get_x_sample(t): 
    x_sample = []
    for i in range(t):
        x_sample.append(np.random.uniform(0,1, x.shape))
        
    return x_sample

In [51]:
x_sample = get_x_sample(t)

def F_fixed(x, f, t, x_sample):
    # Here, we do not re-sample from x every time we run the function
    # Instead, we take a collection of x samples and compute the estimated expectation
    
    sum_R = 0
    for i in range(t):
        R_t = x >= x_sample[i]
        sum_R = sum_R + f(R_t)
    return sum_R/t
                 
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))
print(F_fixed(x, f_linear, 100, x_sample))

-1.0633340141700303
-1.0633340141700303
-1.0633340141700303


## Function to calculate gradients

In [81]:
def get_gradient_F(F,x,f,t,n,x_sample):
    # A vectorized function to get gradients
    
    x_new_1 = x*np.ones((n, n))
    x_new_0 = x*np.ones((n, n))

    np.fill_diagonal(x_new_1, 1)
    np.fill_diagonal(x_new_0, 0)
    
    return F(x_new_1, f, t, x_sample) - F(x_new_0, f, t, x_sample)

print(get_gradient_F(F_fixed,x,f_linear,t,n,x_sample))

[-0.73569541  0.62903284  0.26188165  0.63377371  0.44822793  0.70714339
 -0.16722055 -0.33568168 -0.29152329  0.56592234]


## Gradient ascent

In [160]:
def gradient_ascent(F, x, f, alpha, t, epsilon, n, n_step_max, grad_decay = False, input_x_sample = None):  
    
    if input_x_sample == None:
        x_sample = get_x_sample(t)
    else:
        x_sample = input_x_sample
    
    x = copy.deepcopy(x)
    x_init = copy.deepcopy(x)
    sum_init = F(x, f, t, x_sample)
    
    # key values to be used
    sum_update = 0
    step = 0
    
    sum_temp = copy.deepcopy(sum_init)
    
    # start updating the parameters x with iterative gradients
    while (np.abs(sum_temp - sum_update) > epsilon) & (step < n_step_max) :    
#     while (np.abs(sum_temp - sum_update) > epsilon) & (sum_update > sum_temp) & (step < n_step_max) :
  
        step += 1
        sum_temp = F(x, f, t, x_sample)
        x_temp = x
        
        grad = get_gradient_F(F,x,f,t,n, x_sample)
        
        x = x + alpha*grad
        x = np.maximum(np.minimum(x,1),0)
        
        sum_update = F(x, f, t, x_sample)
        
        if grad_decay == True:
            alpha *= (1/step)**(1/5)
            print("alpha: ", alpha, '\n')
        
        print("Iteration: ", step, "\n" , 
              "Gradient:  ", grad, "\n",
              "Updated  value: ", sum_update,"\n", 
              "Previous value: ", sum_temp, "\n",
              "x: ", x, "\n")
    
        
    
    return step,sum_update, x, x_temp, x_sample

# Testing with a linear function

In [175]:
###
# parameters
###

# coefficients for the linear function
a = np.random.uniform(-1,1, n)

# parameters fo the gradient ascent function
alpha = 0.01
epsilon = 10**(-8)
n_step_max = 5000

# number of iterations for estimating the expectation
t = 300

# number of products in the assortment
n = 10

# initialize the x vector
x_initial = np.random.uniform(0,1, n)
print(x_initial)

[0.67441078 0.13572791 0.76219267 0.8738543  0.66134901 0.6636263
 0.14091129 0.58055908 0.49889681 0.71107235]


### Testing for different values of a

In [176]:
# Use function F_fixed: sampling for estimating expectation is only done once and for all
print(a)
# x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)

[-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963]
Iteration:  1 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.1540484604491951 
 Previous value:  0.12499687501379085 
 x:  [0.67011358 0.12872339 0.75286812 0.87867787 0.65673016 0.67357203
 0.14348184 0.58370281 0.4945277  0.71802915] 

Iteration:  2 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.1953718795540203 
 Previous value:  0.1540484604491951 
 x:  [0.66581639 0.12171887 0.74354356 0.88350144 0.65211131 0.68351775
 0.14605239 0.58684654 0.4901586  0.72498594] 

Iteration:  3 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  0.27551975

Iteration:  34 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.3183112401562311 
 Previous value:  1.2990739243063822 
 x:  [0.52830604 0.         0.44515768 1.         0.50430804 1.
 0.22831    0.68744599 0.35034731 0.94760342] 

Iteration:  35 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.3238115037316398 
 Previous value:  1.3183112401562311 
 x:  [0.52400884 0.         0.43583312 1.         0.49968919 1.
 0.23088055 0.69058972 0.3459782  0.95456022] 

Iteration:  36 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.3537910677593479 
 Previous value:  1.3238115037316398 
 x:  [0.51971165 0.         0.42650856 1.         0.49507034 1.
 0.2334511  0.69373345 0.

Iteration:  79 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.127513720020757 
 Previous value:  2.1075444043316196 
 x:  [0.33493212 0.         0.02555253 1.         0.2964597  1.
 0.34398476 0.82891396 0.15373767 1.        ] 

Iteration:  80 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.1546201631604327 
 Previous value:  2.127513720020757 
 x:  [0.33063492 0.         0.01622797 1.         0.29184085 1.
 0.34655531 0.83205769 0.14936857 1.        ] 

Iteration:  81 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.1673847101504133 
 Previous value:  2.1546201631604327 
 x:  [0.32633772 0.         0.00690342 1.         0.287222   1.
 0.34912586 0.83520143 0.14

Iteration:  107 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.36408586617187 
 Previous value:  2.3610066314583125 
 x:  [0.21461057 0.         0.         1.         0.16713184 1.
 0.41596017 0.91693848 0.03140279 1.        ] 

Iteration:  108 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.37581205726416 
 Previous value:  2.36408586617187 
 x:  [0.21031337 0.         0.         1.         0.16251299 1.
 0.41853072 0.92008221 0.02703369 1.        ] 

Iteration:  109 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.388502316388579 
 Previous value:  2.37581205726416 
 x:  [0.20601617 0.         0.         1.         0.15789414 1.
 0.42110127 0.92322594 0.022664

Iteration:  133 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.542477341367255 
 Previous value:  2.533473417945288 
 x:  [0.10288341 0.         0.         1.         0.04704169 1.
 0.48279448 0.99867553 0.         1.        ] 

Iteration:  134 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.5481441043504724 
 Previous value:  2.542477341367255 
 x:  [0.09858621 0.         0.         1.         0.04242284 1.
 0.48536503 1.         0.         1.        ] 

Iteration:  135 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  2.552548520557768 
 Previous value:  2.5481441043504724 
 x:  [0.09428902 0.         0.         1.         0.03780399 1.
 0.48793558 1.         0. 

#### Check that the last two iterations actually converged

In [184]:
x_previous = x_temp_
print("x (2nd last iter): ", x_previous)
print("Value of F (2nd last iter): ", F_fixed(x_previous, f_linear, t, x_sample_), "\n")

x_updated  = x_
print("x (last iter): ", x_updated)
print("Value of F (last iter): ", F_fixed(x_updated, f_linear, t, x_sample_))

x (2nd last iter):  [0.04701983 0.         0.         1.         0.         1.
 0.51621163 1.         0.         1.        ]
Value of F (2nd last iter):  2.593448092704655 

x (last iter):  [0.04272264 0.         0.         1.         0.         1.
 0.51878218 1.         0.         1.        ]
Value of F (last iter):  2.593448092704655


#### If we manually converge probs to 0 or 1...

In [185]:
x_previous_modified = copy.deepcopy(x_previous)
x_previous_modified[0] = 0
x_previous_modified[6] = 1

print(x_previous_modified)
F_fixed(x_previous_modified, f_linear, t, x_sample_) 

[0. 0. 0. 1. 0. 1. 1. 1. 0. 1.]


2.7440371484995003

#### The convergence will be actually much faster if we use decaying gradient here
Due to the fact that there is no local maximum in the linear function.
And that decreasing in steps will make the incremental change really small.

In [190]:
step_,sum_update_, x_, x_temp_, x_sample_  = gradient_ascent(F_fixed, x, f_linear, 
                                                             alpha, t, epsilon, n, n_step_max, 
                                                             grad_decay=True, input_x_sample=x_sample)

alpha:  0.01 

Iteration:  1 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.7998592062024381 
 Previous value:  1.7890233626199783 
 x:  [0.19415674 0.         0.4869247  0.69133895 0.07749761 1.
 0.38439678 1.         0.         0.9359344 ] 

alpha:  0.008705505632961241 

Iteration:  2 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.8119906607309144 
 Previous value:  1.7998592062024381 
 x:  [0.18985954 0.         0.47760014 0.69616252 0.07287876 1.
 0.38696733 1.         0.         0.94289119] 

alpha:  0.0069882711877157925 

Iteration:  3 
 Gradient:   [-0.42971983 -0.70045157 -0.93245588  0.48235705 -0.46188521  0.99457216
  0.25705503  0.31437327 -0.4369103   0.69567963] 
 Updated  value:  1.828788931001961 
 Previous value:  1.8119906607309144 
 x:  [0.1861186

#### Initialization on x matters

In [ ]:
# x_initial = np.random.uniform(0,1,n)
step_,sum_update_, x_, x_temp_, x_sample_ = gradient_ascent(F_fixed, x_initial, f_linear, alpha, t, epsilon, n, 
                                                            n_step_max,grad_decay=False, input_x_sample = x_sample)

In [320]:
# Use function F: re-sampling is enabled in each iteration

# Note: in this case, the value of the function could decrease as each time the estimated value of F is different 
#       due to differnt samples from x
gradient_ascent(F, x_initial, f_linear, alpha, t, epsilon, n, n_step_max, grad_decay=False)

Iteration:  1 
 Function value:  0.8226142902459623 
 x:  [0.28952444 0.01171267 0.4641246  0.69771235 0.09814708 0.90005682
 0.25291156 0.96392139 0.42386655 0.6628938 ] 

Iteration:  2 
 Function value:  0.8922167709145818 
 x:  [0.28808696 0.00988033 0.46467124 0.69771151 0.09805355 0.90319879
 0.25485543 0.97037754 0.4156565  0.66746391] 

Iteration:  3 
 Function value:  0.9250194095471035 
 x:  [0.2865141  0.00804109 0.46578976 0.69747884 0.09828021 0.90641137
 0.25716688 0.97758802 0.40726102 0.67207145] 

Iteration:  4 
 Function value:  0.917132541431824 
 x:  [0.28505296 0.00658648 0.46609527 0.69724457 0.09786904 0.90941289
 0.25985408 0.98377719 0.39885355 0.67653576] 

Iteration:  5 
 Function value:  0.9648492212778369 
 x:  [0.28324303 0.00467979 0.46677709 0.69739784 0.09792327 0.91341925
 0.26227646 0.99000214 0.39050015 0.68183382] 

Iteration:  6 
 Function value:  0.9756471927897269 
 x:  [0.28120191 0.00356505 0.46693031 0.69705271 0.09737507 0.91648362
 0.26479017

Iteration:  52 
 Function value:  1.4237422390992882 
 x:  [0.20725037 0.         0.49453365 0.68854433 0.08275204 1.
 0.3648116  1.         0.00259928 0.89599132] 

Iteration:  53 
 Function value:  1.4313855762499241 
 x:  [0.20607093 0.         0.49498398 0.6882669  0.0822678  1.
 0.36700211 1.         0.         0.90046016] 

Iteration:  54 
 Function value:  1.4326702629695167 
 x:  [0.2044884  0.         0.4956803  0.68805899 0.08216474 1.
 0.36934732 1.         0.         0.90509008] 

Iteration:  55 
 Function value:  1.4575942460343962 
 x:  [0.20290988 0.         0.4964004  0.68815041 0.08170886 1.
 0.3718364  1.         0.         0.90963842] 

Iteration:  56 
 Function value:  1.444530004737863 
 x:  [0.2015712  0.         0.49668489 0.68799014 0.08152573 1.
 0.3741752  1.         0.         0.91427043] 

Iteration:  57 
 Function value:  1.4700544246591079 
 x:  [0.1999036  0.         0.4971926  0.68769062 0.08106949 1.
 0.37645844 1.         0.         0.91882475] 

Itera

Iteration:  107 
 Function value:  1.5251269094870883 
 x:  [0.12158593 0.         0.52453506 0.67878304 0.06781641 1.
 0.48752187 1.         0.         1.        ] 

Iteration:  108 
 Function value:  1.5235074273812357 
 x:  [0.120114   0.         0.52509969 0.6785359  0.06771471 1.
 0.48977433 1.         0.         1.        ] 

Iteration:  109 
 Function value:  1.5212115407403553 
 x:  [0.11860686 0.         0.52555268 0.67824098 0.06752675 1.
 0.49196385 1.         0.         1.        ] 

Iteration:  110 
 Function value:  1.5273152871494298 
 x:  [0.11725247 0.         0.52608931 0.67779144 0.06738208 1.
 0.49419618 1.         0.         1.        ] 

Iteration:  111 
 Function value:  1.5306849153359556 
 x:  [0.11568055 0.         0.52644418 0.67772674 0.06688324 1.
 0.49635057 1.         0.         1.        ] 

Iteration:  112 
 Function value:  1.527739638507797 
 x:  [0.11411977 0.         0.52695689 0.6775791  0.06674832 1.
 0.49857892 1.         0.         1.        ] 


Iteration:  157 
 Function value:  1.5731455721556211 
 x:  [0.04371808 0.         0.55202124 0.67081108 0.05372751 1.
 0.59813361 1.         0.         1.        ] 

Iteration:  158 
 Function value:  1.5484636224055355 
 x:  [0.04213298 0.         0.55261105 0.67051318 0.05345905 1.
 0.60036434 1.         0.         1.        ] 

Iteration:  159 
 Function value:  1.570775531014238 
 x:  [0.04046408 0.         0.55294768 0.67024194 0.05310275 1.
 0.60253672 1.         0.         1.        ] 

Iteration:  160 
 Function value:  1.5650869527247506 
 x:  [0.03882112 0.         0.55362394 0.67005371 0.05296684 1.
 0.60470516 1.         0.         1.        ] 

Iteration:  161 
 Function value:  1.5648029579008578 
 x:  [0.03712321 0.         0.55422842 0.66986883 0.05287325 1.
 0.60691737 1.         0.         1.        ] 

Iteration:  162 
 Function value:  1.5785068906698216 
 x:  [0.03557822 0.         0.55489534 0.66969073 0.05249642 1.
 0.60913341 1.         0.         1.        ] 


Iteration:  221 
 Function value:  1.6040182796365292 
 x:  [0.         0.         0.58822232 0.65893683 0.03656468 1.
 0.73966274 1.         0.         1.        ] 

Iteration:  222 
 Function value:  1.6043692170844421 
 x:  [0.         0.         0.58874015 0.65868498 0.03627695 1.
 0.74187881 1.         0.         1.        ] 

Iteration:  223 
 Function value:  1.6009696925250572 
 x:  [0.         0.         0.5893614  0.65854469 0.03609466 1.
 0.74409753 1.         0.         1.        ] 

Iteration:  224 
 Function value:  1.6060065433692523 
 x:  [0.         0.         0.58985554 0.65834458 0.03585105 1.
 0.74629725 1.         0.         1.        ] 

Iteration:  225 
 Function value:  1.6183976591304867 
 x:  [0.         0.         0.59029765 0.6581155  0.03555744 1.
 0.74851065 1.         0.         1.        ] 

Iteration:  226 
 Function value:  1.598680886831291 
 x:  [0.         0.         0.59077684 0.65788146 0.03529939 1.
 0.75072711 1.         0.         1.        ] 


...

Iteration:  844 
 Function value:  1.6829878646164047 
 x:  [0.         0.         0.92843837 0.54350096 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  845 
 Function value:  1.6827663371548756 
 x:  [0.         0.         0.92897439 0.54330438 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  846 
 Function value:  1.6823820722589065 
 x:  [0.         0.         0.92951475 0.54312421 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  847 
 Function value:  1.6834145179518365 
 x:  [0.         0.         0.93006315 0.54293128 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  848 
 Function value:  1.6844045752053038 
 x:  [0.         0.         0.93061588 0.54273106 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  849 
 Function value:  1.682759840441915 
 x:  [0.         0.         0.93116552 0.54256001 0.         1.
 1.         1.         0.         1.        ] 


Iteration:  896 
 Function value:  1.683600153686862 
 x:  [0.         0.         0.95702322 0.53384243 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  897 
 Function value:  1.6848866104061186 
 x:  [0.         0.         0.95757472 0.53366044 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  898 
 Function value:  1.6853816390328513 
 x:  [0.         0.         0.95812312 0.53348027 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  899 
 Function value:  1.6851960032978266 
 x:  [0.         0.         0.95867337 0.53329464 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  900 
 Function value:  1.6856161599202995 
 x:  [0.         0.         0.9592292  0.53309988 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  901 
 Function value:  1.6838379229307896 
 x:  [0.         0.         0.95976831 0.53290513 0.         1.
 1.         1.         0.         1.        ] 


 Function value:  1.6848703686237205 
 x:  [0.         0.         0.98990307 0.52267145 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  957 
 Function value:  1.6868896232136381 
 x:  [0.         0.         0.99044281 0.52249494 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  958 
 Function value:  1.6867723627699143 
 x:  [0.         0.         0.99099554 0.5223093  0.         1.
 1.         1.         0.         1.        ] 

Iteration:  959 
 Function value:  1.6874465304187123 
 x:  [0.         0.         0.99154889 0.52211819 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  960 
 Function value:  1.6868928715701172 
 x:  [0.         0.         0.99208925 0.52193621 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  961 
 Function value:  1.6870817556616222 
 x:  [0.         0.         0.99264321 0.52175057 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  962

Iteration:  1006 
 Function value:  1.6882477035806744 
 x:  [0.         0.         1.         0.51339879 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  1007 
 Function value:  1.6868863748571585 
 x:  [0.         0.         1.         0.51321315 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  1008 
 Function value:  1.687195767748866 
 x:  [0.         0.         1.         0.51302751 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  1009 
 Function value:  1.6878764321106245 
 x:  [0.         0.         1.         0.51284188 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  1010 
 Function value:  1.687072010592183 
 x:  [0.         0.         1.         0.51265624 0.         1.
 1.         1.         0.         1.        ] 

Iteration:  1011 
 Function value:  1.6870720105921828 
 x:  [0.         0.         1.         0.51247061 0.         1.
 1.         1.         0.         1.      

(1035,
 1.6875051606405749,
 array([0.        , 0.        , 1.        , 0.50801535, 0.        ,
        1.        , 1.        , 1.        , 0.        , 1.        ]))